In [187]:
import glob, os
import pandas as pd
import pdb
import subprocess
from IPython.core import display as ICD
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
import numpy as np

In [5]:
bids_path = '/projects/niblab/bids_projects/Experiments/bbx/bids/'

Expected subject range:  1 - 158


In [45]:
# get scan notes
df_w1_notes=pd.read_csv('w1_notes.csv', encoding='latin-1')
#df_w1_notes.head()
#df_w1_notes.columns
df_clean=df_w1_notes[['participantID', 'w1scan_scannotes']]

df_clean.set_index("participantID", inplace=True)
df_clean.index = df_clean.index.str.lower()
df_clean = df_clean.drop(['participant id (bbx_###)'])

## DICOM Check

In [120]:

# Get DICOM id list
s1_dcms = [x.split("/")[-1].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sourcedata/DICOM/ses-1/sub-*"))]
s1_dcms = np.unique(np.array(s1_dcms)).tolist()
# get expected id list from notes
s1_sub_ids = sorted([x.split("_")[1].lstrip('0') for x in df_clean.index.values])
s1_sub_ids = np.unique(np.array(s1_sub_ids)).tolist()

# get total count for DICOM and expected id lists
s1_sub_exp_ct = len(s1_sub_ids)
s1_sub_dcm_ct = len(s1_dcms)

# return the unique values in ar1 that are not in ar2
# ids found in dicom directory but not id list
s1_mia_id = np.setdiff1d(s1_dcms, s1_sub_ids)

# ids missing from dicom directories
s1_mia_dcm = np.setdiff1d(s1_sub_id_lst, s1_dcms)

s1_mia_id = s1_mia_id.tolist()
s1_mia_dcm = s1_mia_dcm.tolist()

# look at scan notes for any missing DICOM ids
for id_ in s1_mia_dcm:
    bbx_id = "bbx_{:03d}".format(int(id_))
    try:
        print("{}, notes: \n{}".format(bbx_id, df_clean.loc[bbx_id]))
    except:
        print("Missing scan notes for ", bbx_id)

## Report Output
print("\nExpected subject count: {} \tUnique DICOM directories found: {}".format(s1_sub_exp_ct, s1_sub_dcm_ct))
print("\nMissing DICOM directories for IDs: {} \n\nMissing scan note IDs, but found DICOM directories: {} \n".format(s1_mia_dcm, s1_mia_id))




Expected subject count: 142 	Unique DICOM directories found: 144

Missing DICOM directories for IDs: [] 

Missing scan note IDs, but found DICOM directories: ['154', '155'] 



## BIDS Check  
Other:    
* BIDS validator  
* Volume Check
#### Compare DICOM ids with BIDS ids found, list any missing from BIDS

In [252]:
s1_bids = np.unique(np.array([x.split("/")[-2].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sub-*/ses-1"))])).tolist()
s1_bids_ct = len(s1_bids)

# subjects found in DICOM list but not in bids
s1_mia_bids = np.setdiff1d(s1_dcms, s1_bids)

print("Unique BIDS subject count: {} \nMissing IDs: \t{} \n".format(s1_bids_ct, s1_mia_bids))


Unique BIDS subject count: 144 
Missing IDs: 	[] 



#### Fill in a dictionary with subject ids and find their directories:  
#### *Value of 1 translates that the directory and files are missing*

In [253]:

# if no missing IDs found, that is good, meaning we have at least attempted to generate all of the subject directories available
# now we have to inspect individual directories
qa_dict={}
bids_dirs = glob.glob(os.path.join(bids_path, "sub-*/ses-1"))
for dir_path in sorted(bids_dirs):
    sub_id = dir_path.split("/")[-2]
    if sub_id not in qa_dict:
        qa_dict[sub_id] = {
            "FUNC": 0,
            "FMAP": 0,
            "ANAT": 0
        }
        
    funcs=glob.glob(os.path.join(dir_path, "func/*"))
    anat=glob.glob(os.path.join(dir_path, "fmap/*"))
    fmaps=glob.glob(os.path.join(dir_path, "anat/*"))
    #print("Analysis of {}:".format(sub_id))
    if not funcs:
        qa_dict[sub_id]["FUNC"] = 1
    if not fmaps:
        qa_dict[sub_id]["FMAP"] = 1
    if not anat:
        qa_dict[sub_id]["ANAT"] = 1
    
    
    

In [254]:
bids_df = pd.DataFrame(qa_dict).T

In [255]:
missing_df = bids_df[bids_df.any(axis="columns")]

In [256]:
ICD.display(missing_df.head(20))

,FUNC,FMAP,ANAT
sub-025,1,0,1
sub-094,0,0,1
sub-104,1,0,1
sub-115,1,0,0
sub-117,1,0,0
sub-128,0,0,1
sub-134,1,0,1
sub-155,1,0,0


In [257]:
bids_ids_mia=list(missing_df.index.values)

In [258]:
!module load tree/1.8.0

In [259]:

for sub_id in missing_df.index.values:
    print("\nID: ",sub_id)
    dir_path = glob.glob(os.path.join(bids_path, sub_id, "ses-1/*"))
    print("\nFiles Found: ")
    for z in dir_path:
        print(z)
    try:
        print("\nDIRECTORIES FOUND MISSING: ")
        ICD.display( missing_df.loc[sub_id])
        print("SCAN NOTES (IF AVAILABLE):")
        ICD.display( df_clean.loc[sub_id.replace("sub-", "bbx_")])
    except:
        print("\nNo scan notes found")
    print("____________________________________")


ID:  sub-025

Files Found: 
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-025/ses-1/anat
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-025/ses-1/sub-025_ses-1_scans.tsv

DIRECTORIES FOUND MISSING: 


FUNC    1
FMAP    0
ANAT    1
Name: sub-025, dtype: int64

SCAN NOTES (IF AVAILABLE):

No scan notes found
____________________________________

ID:  sub-094

Files Found: 
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-094/ses-1/anat
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-094/ses-1/sub-094_ses-1_scans.tsv
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-094/ses-1/func

DIRECTORIES FOUND MISSING: 


FUNC    0
FMAP    0
ANAT    1
Name: sub-094, dtype: int64

SCAN NOTES (IF AVAILABLE):


...

SCAN NOTES (IF AVAILABLE):


w1scan_scannotes    Excellent job after redoing the anatomical 
Name: bbx_134, dtype: object

____________________________________

ID:  sub-155

Files Found: 
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-155/ses-1/anat
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-155/ses-1/sub-155_ses-1_scans.tsv
/projects/niblab/bids_projects/Experiments/bbx/bids/sub-155/ses-1/fmap

DIRECTORIES FOUND MISSING: 


FUNC    1
FMAP    0
ANAT    0
Name: sub-155, dtype: int64

SCAN NOTES (IF AVAILABLE):

No scan notes found
____________________________________


### Notes:  
* sub-025 might have to be eliminated, will not run through bids further inspection needed.  

In [260]:
bids_ids_mia

['sub-025',
 'sub-094',
 'sub-104',
 'sub-115',
 'sub-117',
 'sub-128',
 'sub-134',
 'sub-155']